<a href="https://colab.research.google.com/github/yagamiAbhi/Home-Loan-Approval-System-using-pySpark/blob/main/Home_Loan_Approval_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Consider one of the leading financial organization that provides Health Insurance, Home loan, Education Loan, and other banking services globally to its customers.


Approving an online home loan application is critical as it requires proper verification and validation of the user information. To provide optimized user experience, organization has decided to provide instant online home loan approval.


The technical team is using HDFS for storage, Spark core and Spark SQL to process online home loan requests and provide faster approval.


In the approval process, below business rules to be verified.



1.   Customer’s defaulter status
2.   Customer’s credit score which provides an insight for the eligibility
     of approving a home loan application




In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=a78b4ea943207ea7592c1566b9fef940891ddc37201b78cbd60ebc62e44200e9
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
from pyspark import SparkContext
sc = SparkContext(master='local',appName='HLAS')

In [3]:
from pyspark.sql import SparkSession, SQLContext
spark = SparkSession(sc)
sqlContext = SQLContext(sc)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


### Task 1: Create Spark SQL DataFrames

In [4]:
applicationDataRDD = sc.textFile('/content/HomeLoanApplicationData.csv')
ReferenceDataRDD = sc.textFile('/content/ClientReferenceDataset.csv')

In [5]:
applicationDataRDD = applicationDataRDD.map(lambda x: x.split(','))
ReferenceDataRDD = ReferenceDataRDD.map(lambda x: x.split(','))

In [6]:
applicationDataRDD.take(2)


[['Abhishek',
  '23-08-81',
  'UIN0001',
  'Abhishek@mail.com',
  '1287655553',
  'Ahmedabad',
  'Gujarat',
  'APL',
  '380001',
  '23000'],
 ['Aman',
  '17-09-84',
  'UIN0002',
  'Aman@mail.com',
  '9776543221',
  'Mumbai',
  'Maharashtra',
  'BPL',
  '400001',
  '22500']]

In [7]:
from pyspark.sql.types import StructType, StructField, StringType

app_schema = StructType([StructField('CustomerName', StringType(), True),
                        StructField('DOB', StringType(), True),
                        StructField('UIN', StringType(), True),
                        StructField('MailID', StringType(), True),
                        StructField('PhoneNumber', StringType(), True),
                        StructField('City', StringType(), True),
                        StructField('State', StringType(), True),
                        StructField('LivingStatus', StringType(), True),
                        StructField('PinCode', StringType(), True),
                        StructField('LoanAmount', StringType(), True)]
                        )

ref_schema = StructType([StructField('CustomerName', StringType(), True),
                        StructField('DOB', StringType(), True),
                        StructField('UIN', StringType(), True),
                        StructField('City', StringType(), True),
                        StructField('State', StringType(), True),
                        StructField('PinCode', StringType(), True),
                        StructField('CibilScore', StringType(), True),
                        StructField('DefaulterFlag', StringType(), True)]
                        )

In [18]:
application_df = spark.createDataFrame(applicationDataRDD, app_schema)

In [52]:
from pyspark.sql.functions import col, lit, when

In [20]:
application_df = application_df.withColumn('PhoneNumber', col('PhoneNumber').cast('long')).withColumn('LoanAmount', col('LoanAmount').cast('long'))

In [21]:
application_df.printSchema()

root
 |-- CustomerName: string (nullable = true)
 |-- DOB: string (nullable = true)
 |-- UIN: string (nullable = true)
 |-- MailID: string (nullable = true)
 |-- PhoneNumber: long (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- LivingStatus: string (nullable = true)
 |-- PinCode: string (nullable = true)
 |-- LoanAmount: long (nullable = true)



In [23]:
reference_df = ReferenceDataRDD.toDF(ref_schema)
reference_df = reference_df.withColumn('PinCode', reference_df['PinCode'].cast('long')).withColumn('CibilScore', reference_df['CibilScore'].cast('long'))
reference_df.printSchema()

root
 |-- CustomerName: string (nullable = true)
 |-- DOB: string (nullable = true)
 |-- UIN: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- PinCode: long (nullable = true)
 |-- CibilScore: long (nullable = true)
 |-- DefaulterFlag: string (nullable = true)



In [24]:
reference_df.show(3)

+------------+--------+-------+-----------------+-----------+-------+----------+-------------+
|CustomerName|     DOB|    UIN|             City|      State|PinCode|CibilScore|DefaulterFlag|
+------------+--------+-------+-----------------+-----------+-------+----------+-------------+
|    Abhishek|23-08-81|UIN0001|        Ahmedabad|    Gujarat| 380001|       910|            N|
|        Aman|17-09-84|UIN0002|           Mumbai|Maharashtra| 400001|       910|            N|
|       Harsh|17-03-83|UIN0003|Thiruvananthpuram|     Kerala| 695001|       530|            N|
+------------+--------+-------+-----------------+-----------+-------+----------+-------------+
only showing top 3 rows



In [25]:
application_df.show(3)

+------------+--------+-------+-----------------+-----------+-----------------+-----------+------------+-------+----------+
|CustomerName|     DOB|    UIN|           MailID|PhoneNumber|             City|      State|LivingStatus|PinCode|LoanAmount|
+------------+--------+-------+-----------------+-----------+-----------------+-----------+------------+-------+----------+
|    Abhishek|23-08-81|UIN0001|Abhishek@mail.com| 1287655553|        Ahmedabad|    Gujarat|         APL| 380001|     23000|
|        Aman|17-09-84|UIN0002|    Aman@mail.com| 9776543221|           Mumbai|Maharashtra|         BPL| 400001|     22500|
|       Harsh|17-03-83|UIN0003|   Harsh@mail.com| 9987765234|Thiruvananthpuram|     Kerala|         BPL| 695001|     21500|
+------------+--------+-------+-----------------+-----------+-----------------+-----------+------------+-------+----------+
only showing top 3 rows



### Task 2: Joining DataFrames

In [37]:
join_data = application_df.join(other=reference_df, on=['UIN','CustomerName','DOB','City','State','PinCode'], how='inner').sort('UIN', ascending=True)
join_data.show(3)

+-------+------------+--------+-----------------+-----------+-------+-----------------+-----------+------------+----------+----------+-------------+
|    UIN|CustomerName|     DOB|             City|      State|PinCode|           MailID|PhoneNumber|LivingStatus|LoanAmount|CibilScore|DefaulterFlag|
+-------+------------+--------+-----------------+-----------+-------+-----------------+-----------+------------+----------+----------+-------------+
|UIN0001|    Abhishek|23-08-81|        Ahmedabad|    Gujarat| 380001|Abhishek@mail.com| 1287655553|         APL|     23000|       910|            N|
|UIN0002|        Aman|17-09-84|           Mumbai|Maharashtra| 400001|    Aman@mail.com| 9776543221|         BPL|     22500|       910|            N|
|UIN0003|       Harsh|17-03-83|Thiruvananthpuram|     Kerala| 695001|   Harsh@mail.com| 9987765234|         BPL|     21500|       530|            N|
+-------+------------+--------+-----------------+-----------+-------+-----------------+-----------+-------

In [38]:
print("Total records in app_df is : ",application_df.count())
print("Total records in ref_df is : ",reference_df.count())
print("Total records in join_df is : ",join_data.count())

Total records in app_df is :  60
Total records in ref_df is :  60
Total records in join_df is :  60


### 1.   Customers with defaulter status





In [40]:
defaulter = join_data.filter('DefaulterFlag=="Y"')

In [42]:
print('Total numbers of customers with defaulter statuts is :', defaulter.count())
defaulter.show()

Total numbers of customers with defaulter statuts is : 14
+-------+------------+--------+-----------------+-----------+-------+-----------------+-----------+------------+----------+----------+-------------+
|    UIN|CustomerName|     DOB|             City|      State|PinCode|           MailID|PhoneNumber|LivingStatus|LoanAmount|CibilScore|DefaulterFlag|
+-------+------------+--------+-----------------+-----------+-------+-----------------+-----------+------------+----------+----------+-------------+
|UIN0005|       Aditi|21-08-79|           Mumbai|Maharashtra| 400001|   Aditi@mail.com| 8888777722|         BPL|     25500|      1300|            Y|
|UIN0008|     Anushka|25-08-82|Thiruvananthpuram|     Kerala| 695001| Anushka@mail.com| 3344990876|         APL|     24500|      1530|            Y|
|UIN0010|     Saurabh|22-08-83|           Mumbai|Maharashtra| 400001| Saurabh@mail.com| 3344990876|         APL|     20500|      4530|            Y|
|UIN0020|       Kajal|22-08-85|           Mumbai

### 2.   customers whose LivingStatus is “BPL” or “APL


In [45]:
cust_bpl_apl = join_data.filter( "LivingStatus=='BPL'" or "LivingStatus=='APL'" )

In [46]:
print('Total numbers of customers whose LivingStatus is “BPL” or “APL is :', cust_bpl_apl.count())

Total numbers of customers whose LivingStatus is “BPL” or “APL is : 28


### 3.   Apply the status to be Approved if the client is not a defaulter and the credit score is more than 800




In [60]:
final_data = join_data.withColumn('Status', when( (join_data['DefaulterFlag'] == 'N') &
 (join_data['CibilScore']>800), "Approved").otherwise("Rejected"))

In [61]:
final_data.show()

+-------+------------+--------+-----------------+-----------+-------+-----------------+-----------+------------+----------+----------+-------------+--------+
|    UIN|CustomerName|     DOB|             City|      State|PinCode|           MailID|PhoneNumber|LivingStatus|LoanAmount|CibilScore|DefaulterFlag|  Status|
+-------+------------+--------+-----------------+-----------+-------+-----------------+-----------+------------+----------+----------+-------------+--------+
|UIN0001|    Abhishek|23-08-81|        Ahmedabad|    Gujarat| 380001|Abhishek@mail.com| 1287655553|         APL|     23000|       910|            N|Approved|
|UIN0002|        Aman|17-09-84|           Mumbai|Maharashtra| 400001|    Aman@mail.com| 9776543221|         BPL|     22500|       910|            N|Approved|
|UIN0003|       Harsh|17-03-83|Thiruvananthpuram|     Kerala| 695001|   Harsh@mail.com| 9987765234|         BPL|     21500|       530|            N|Rejected|
|UIN0004|       Ayush|22-02-78|Thiruvananthpuram|   

### 4.   Display the UIN, Client Name, Status & loan amount

In [63]:
final_file = final_data.select('UIN','CustomerName','Status','LoanAmount')
final_file.show()

+-------+------------+--------+----------+
|    UIN|CustomerName|  Status|LoanAmount|
+-------+------------+--------+----------+
|UIN0001|    Abhishek|Approved|     23000|
|UIN0002|        Aman|Approved|     22500|
|UIN0003|       Harsh|Rejected|     21500|
|UIN0004|       Ayush|Rejected|     21500|
|UIN0005|       Aditi|Rejected|     25500|
|UIN0006|      Anjali|Approved|     21000|
|UIN0007|     Shubham|Approved|     21000|
|UIN0008|     Anushka|Rejected|     24500|
|UIN0009|       Rohit|Approved|     22000|
|UIN0010|     Saurabh|Rejected|     20500|
|UIN0011|      Muskan|Rejected|     20500|
|UIN0012|       Rahul|Rejected|     23500|
|UIN0013|     Utkarsh|Rejected|     23000|
|UIN0014|     Vaibhav|Rejected|     24500|
|UIN0015|        Amit|Rejected|     23500|
|UIN0016|      Saumya|Rejected|     24500|
|UIN0017|     Rishabh|Rejected|     23000|
|UIN0018|      Shruti|Rejected|     23000|
|UIN0019|    Himanshu|Rejected|     24500|
|UIN0020|       Kajal|Rejected|     20500|
+-------+--

### 4. Moving Final data to hive table

In [ ]:
from pyspark.sql import HiveContext
hc = HiveContext(sc)
print(hc)

In [67]:
sqlContext.sql('create database homeloandb')
sqlContext.sql('use homeloandb')
sqlContext.tableNames()

[]

In [68]:
from pyspark.sql import DataFrameWriter
dfw = DataFrameWriter(final_file)
dfw.saveAsTable(name="home_loan_approval",mode='overwrite')

In [69]:
sqlContext.tableNames()

['home_loan_approval']

In [71]:
sql = spark.sql("SELECT * from home_loan_approval WHERE Status = 'Approved' ORDER BY LoanAmount DESC")
sql.show()

+-------+------------+--------+----------+
|    UIN|CustomerName|  Status|LoanAmount|
+-------+------------+--------+----------+
|UIN0001|    Abhishek|Approved|     23000|
|UIN0031|        Yash|Approved|     23000|
|UIN0002|        Aman|Approved|     22500|
|UIN0032|      Shreya|Approved|     22500|
|UIN0009|       Rohit|Approved|     22000|
|UIN0039|       Nidhi|Approved|     22000|
|UIN0006|      Anjali|Approved|     21000|
|UIN0007|     Shubham|Approved|     21000|
|UIN0036|       Swati|Approved|     21000|
|UIN0037|      Vishal|Approved|     21000|
+-------+------------+--------+----------+



### 5. customer's name who are eligible for home loan

Based on:

*   Customer LivingStatus should be “BPL”
*   Customer should not be defaulter
*   Customer Credit score should be more than 800





In [74]:
final_data.filter('LivingStatus=="BPL"' and "Status=='Approved'").select('CustomerName').show()

+------------+
|CustomerName|
+------------+
|    Abhishek|
|        Aman|
|      Anjali|
|     Shubham|
|       Rohit|
|        Yash|
|      Shreya|
|       Swati|
|      Vishal|
|       Nidhi|
+------------+

